# gnomAD v4 Ancestry PCA Projection - Summary

## Objective
Project Tapestry CDS variants (n=97,422) onto gnomAD v4 PC space using official PCA loadings.

## Key Problem: Batch Effects Dominate PC2
**Initial projection showed PC2 stratified by assay version** (v2/v3 vs v4/v6), not ancestry.
- Driven by systematic allele frequency differences between assay batches
- Standard LD pruning insufficient (CDS variants have low background LD)

## Solution: AF-Based Variant Selection
Custom approach: within each LD region, select variant closest to AF = 0.5
- Acts as implicit quality filter (well-called variants)
- Eliminates batch-specific frequency artifacts  
- Preserves ancestry signal (population-differentiating variants)
- Reduced 168,373 → ~80,000 high-quality variants

## Results
- **Batch effects eliminated**: PC2 no longer separates by assay version
- **Clean ancestry structure**: PC1-5 capture population variation
- **Ready for RF classification**: No technical artifacts in top 10 PCs

## Key Insight
**CDS-only variant sets are susceptible to batch effects from multi-assay data.** AF-based selection (targeting variants near 0.5) effectively removes technical artifacts while maintaining biological signal—generalizable strategy for multi-batch genomic datasets where standard LD pruning fails.

## Files Generated
- `gnomad_loadings.txt` - 168,373 variants × 20 PC loadings
- `tapestry_pcs_debatched.sscore` - 97,422 samples × 20 projected PCs
- Diagnostic plots showing before/after batch correction

## Next Step
Apply gnomAD v4 Random Forest classifier to assign ancestry labels (Notebook 04).


## Download gnomAD Reference Data

https://gnomad.broadinstitute.org/data#v4-genetic-ancestry-group-classification

https://console.cloud.google.com/storage/browser/gcp-public-data--gnomad/release/4.0/pca?pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))

In [ ]:
!wget https://storage.googleapis.com/gcp-public-data--gnomad/release/4.0/pca/gnomad.v4.0.pca_loadings.ht /home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD

## Load gnomAD loadings

In [ ]:
import hail as hl
hl.init(
    tmp_dir='/tmp/hail',  # Use system tmp
    local='local[4]',
    quiet=False  # See full error messages
)


In [ ]:
loadings_ht = hl.read_table('/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/gnomad.v4.0.pca_loadings.ht')
loadings_df = loadings_ht.to_pandas()  # Convert to pandas

In [ ]:
loadings_df.to_csv('/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/gnomad_loadings.csv', index=False)

## Extract overlapping variants from Tapestry

In [ ]:
# Create variant list from gnomAD loadings
# Save as chr:pos:ref:alt format (or chr:pos format, check gnomAD structure)
import pandas as pd
loadings_df = pd.read_csv('/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/gnomad_loadings.csv')  # from Step 1
print(loadings_df.shape)
loadings_df.head()


In [ ]:
n_pcs = len(ast.literal_eval(loadings_df['loadings'].values[0]))
print(f'Number of PCs: {n_pcs}')

In [ ]:
import ast
def format_variant_id(row):
    """Convert locus and alleles to chr:pos:ref:alt format"""
    locus = str(row['locus']).replace("chr", "")  # Remove 'chr' prefix if present
    alleles_list = ast.literal_eval(row['alleles'])
    ref = alleles_list[0]
    alt = alleles_list[1]
    return f"{locus}:{ref}:{alt}"

loadings_df['variant_id'] = loadings_df.apply(format_variant_id, axis=1)
loadings_df.head()

In [ ]:
# parse the loadings array
loadings_df['loadings_parsed'] = loadings_df['loadings'].apply(ast.literal_eval)

# Extract PC columns
for i in range(n_pcs):  # gnomAD v4 has 16 PCs
    loadings_df[f'PC{i+1}'] = loadings_df['loadings_parsed'].apply(lambda x: x[i])

# Verify
print(loadings_df[['variant_id', 'PC1', 'PC2', 'PC3']].head())

In [ ]:
variants_path = '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/gnomad_variants.txt'
loadings_df[['variant_id']].to_csv(variants_path, index=False, header=False)

In [ ]:
import subprocess

# Extract these variants from your Tapestry data
# Run plink2 as a subprocess
subprocess.run([
  "plink2",
  "--pfile", "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_wide/genome_qc",
  "--extract", variants_path,
  "--export", "A",
  "--out", "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_overlap/tapestry_gnomad_overlap"
], check=True)

# --export A creates .raw file (additive coding: 0/1/2 genotypes)

## Load Tapestry genotypes and align with loadings

In [ ]:
# Create score file
score_df = loadings_df[['variant_id'] + [f'PC{i}' for i in range(1, 31)]].copy()

# Add allele column (which allele the score applies to)
score_df['allele'] = loadings_df['alleles'].apply(lambda x: ast.literal_eval(x)[1])  # alt allele

# Reorder: variant_id, allele, PC1, PC2, ...
score_df = score_df[['variant_id', 'allele'] + [f'PC{i}' for i in range(1, 31)]]

# Save (tab-separated, no header for plink)
score_path = '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/gnomad_loadings.score'
score_df.to_csv(score_path, sep='\t', index=False, header=False)

In [ ]:
score_df.head()

In [ ]:
subprocess.run([
  "plink2",
  "--pfile", "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_wide/genome_qc",
  "--extract", variants_path,
  "--freq",
  "--out", "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_overlap/tapestry_overlap_freq"
], check=True)

In [ ]:
import pandas as pd

# Read frequency file
freq_df = pd.read_csv('/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_overlap/tapestry_overlap_freq.afreq', sep='\s+')

print(f"Total variants: {len(freq_df)}")

# Function to sum ALT frequencies (handles both biallelic and multiallelic)
def sum_alt_freqs(freq_str):
    """Sum comma-separated frequencies, return total ALT frequency"""
    try:
        freqs = [float(x) for x in str(freq_str).split(',')]
        return sum(freqs)
    except:
        return np.nan

# Calculate total ALT frequency
freq_df['total_alt_freq'] = freq_df['ALT_FREQS'].apply(sum_alt_freqs)

# Check for any parsing errors
if freq_df['total_alt_freq'].isna().any():
    print(f"Warning: {freq_df['total_alt_freq'].isna().sum()} variants failed parsing")
    freq_df = freq_df.dropna(subset=['total_alt_freq'])

# Filter to polymorphic variants (0 < total_ALT_freq < 1)
freq_df_poly = freq_df[(freq_df['total_alt_freq'] > 0) & (freq_df['total_alt_freq'] < 1)]

print(f"Polymorphic variants: {len(freq_df_poly)}")
print(f"Removed {len(freq_df) - len(freq_df_poly)} monomorphic variants")

# Show examples of filtered variants
print("\nExamples of removed variants:")
freq_df_removed = freq_df[~freq_df.index.isin(freq_df_poly.index)]
print(freq_df_removed[['ID', 'ALT_FREQS', 'total_alt_freq']].head(10))

# Save filtered frequency file (for --read-freq)
freq_df_poly[['#CHROM', 'ID', 'REF', 'ALT', 'ALT_FREQS', 'OBS_CT']].to_csv(
    '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/tapestry_overlap_freq_polymorphic.afreq', sep='\t', index=False
)

# Save variant IDs (for --extract)
freq_df_poly['ID'].to_csv('/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/gnomad_variants_polymorphic.txt', index=False, header=False)

print(f"\nSaved: gnomad_variants_polymorphic.txt ({len(freq_df_poly)} variants)")


In [ ]:
import subprocess
score_path = '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/gnomad_loadings.score'
subprocess.run([
  "plink2",
  "--pfile", "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_wide/genome_qc",
  "--extract", '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/gnomad_variants_polymorphic.txt',
  "--read-freq", '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/tapestry_overlap_freq_polymorphic.afreq',
  "--score", score_path, "1", "2", "header",
  "no-mean-imputation",
  "variance-standardize", 
  "cols=+scoresums", 
  "--score-col-nums", "3-32",
  "--out", "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_overlap/tapestry_projected_pcs"
], check=True)

In [ ]:
import pandas as pd
read_path = "/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_overlap/tapestry_projected_pcs.sscore"
pcs = pd.read_csv(read_path, sep='\s+')

# Keep only the AVG columns (these are the projected PCs)
pc_cols = [col for col in pcs.columns if '_SUM' in col and 'DOSAGE' not in col]
pcs_clean = pcs[['#IID'] + pc_cols].copy()

# Rename columns
pcs_clean.columns = ['IID'] + [f'PC{i}' for i in range(1, 31)]

print(f"Projected {len(pcs_clean)} samples onto 30 PCs")
pcs_clean.to_csv('/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/gnomAD/tapestry_projected_pcs.csv', index=False)
pcs_clean.head()

In [ ]:
pcs = pd.read_csv(read_path, sep='\s+')
pcs.columns

## Validation Plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')

# Create figure with multiple subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# PC1 vs PC2 (most informative)
ax1 = axes[0, 0]
ax1.scatter(pcs_clean['PC1'], pcs_clean['PC2'], alpha=0.3, s=5)
ax1.set_xlabel('PC1', fontsize=12)
ax1.set_ylabel('PC2', fontsize=12)
ax1.set_title('PC1 vs PC2', fontsize=14)
ax1.grid(alpha=0.3)

# PC1 vs PC3
ax2 = axes[0, 1]
ax2.scatter(pcs_clean['PC1'], pcs_clean['PC3'], alpha=0.3, s=5)
ax2.set_xlabel('PC1', fontsize=12)
ax2.set_ylabel('PC3', fontsize=12)
ax2.set_title('PC1 vs PC3', fontsize=14)
ax2.grid(alpha=0.3)

# PC2 vs PC3
ax3 = axes[1, 0]
ax3.scatter(pcs_clean['PC2'], pcs_clean['PC3'], alpha=0.3, s=5)
ax3.set_xlabel('PC2', fontsize=12)
ax3.set_ylabel('PC3', fontsize=12)
ax3.set_title('PC2 vs PC3', fontsize=14)
ax3.grid(alpha=0.3)

# PC1 vs PC4
ax4 = axes[1, 1]
ax4.scatter(pcs_clean['PC1'], pcs_clean['PC4'], alpha=0.3, s=5)
ax4.set_xlabel('PC1', fontsize=12)
ax4.set_ylabel('PC4', fontsize=12)
ax4.set_title('PC1 vs PC4', fontsize=14)
ax4.grid(alpha=0.3)

plt.tight_layout()
# plt.savefig('gnomad_projected_pcs_overview.png', dpi=300)
plt.show()